In [1]:
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem as sem
import glob

pd.set_option("display.max_rows", None)

from argparse import ArgumentParser
import itertools as it

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline
from pipeline.utils import validate
from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

adding code to the pythonpath...
[Errno 2] No such file or directory: '/home/anna/anaconda3/envs/biosimspace-dev/bin/pmemd.cuda'
could not determine AMBER version.


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [ ]:
# engine
eng = "GROMACS"

# failed perturbations
failed_perts = []

for protein in ["tyk2", "mcl1", "p38"]:
    bench_folder = f"/home/anna/Documents/benchmark"
    main_dir = f"{bench_folder}/extracted/{protein}"
    # main_dir = f"/backup/42_reruns/{protein}"

    # choose location for the files
    net_file = f"{main_dir}/execution_model/network_combined.dat"
    ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
    exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

    # if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    #     results_folder = f"{main_dir}/outputs_extracted/results"
    # elif os.path.exists(f"{main_dir}/outputs/results"):
    #     results_folder = f"{main_dir}/outputs/results"
    # else:
    #     raise ValueError(
    #         f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline"
    #     )
    results_folder = f"{main_dir}/outputs_extracted/results"
    output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)
    all_analysis_object = analysis_network(
        results_folder,
        exp_file=exp_file,
        net_file=net_file,
        output_folder=output_folder,
        analysis_prot=ana_file,
        # method = "updated",
        engines=eng,
    )

    # can add any other results files
    # all_analysis_object.compute_other_results(file_name=None, name=None)
    all_analysis_object.compute_results(use_cinnabar=True)
    # check if there are any failed perturbations
    failed_perts.append(all_analysis_object.failed_runs(eng))

# flatten perts
failed_perts = [item for row in failed_perts for item in row]

In [ ]:
main_folder = "/home/anna/Documents/benchmark"

df_dict = {}

for prot in ["tyk2", "mcl1", "p38"]:
    file = f"{main_folder}/extracted/{prot}/perturbing_overlap.dat"
    df = pd.read_csv(file)
    df["score"] = np.nan

    perturbations, ligs = pipeline.analysis.get_info_network(
        f"{main_folder}/{prot}_benchmark/execution_model/network_combined.dat"
    )

    # # write lomap scores for all of the network
    # pl = pipeline.setup.initialise_pipeline()
    # # where the ligands for the pipeline are located. These should all be in the same folder in sdf format
    # pl.ligands_folder(f"{main_folder}/inputs/{prot}/ligands")
    # pl.main_folder(f"{main_folder}/{prot}_benchmark")
    # pl.setup_ligands(file_name=f"{main_folder}/{prot}_benchmark/execution_model/combined/ligands.dat")
    # pl.setup_network(folder="combined")
    # for pert in pl.perturbations:
    #     pl.remove_perturbation(pert)
    # for pert in perturbations:
    #     pl.add_perturbation(pert)

    # read in all the lomap scores
    score_dict = {}
    with open(
        f"{main_folder}/{prot}_benchmark/execution_model/network_scores.dat"
    ) as lfile:
        for line in lfile:
            score_dict[
                f"{line.split(',')[0].strip()}~{line.split(',')[1].strip()}"
            ] = float(line.split(",")[-1].strip())

    for index, row in df.iterrows():
        if row["perturbation"] not in perturbations:
            df = df.drop(index)
        else:
            df.at[index, "score"] = score_dict[row["perturbation"]]

    df_dict[prot] = df

frames = []
for key in df_dict.keys():
    frames.append(df_dict[key])

all_df = pd.concat(frames)
df = all_df
# df = df_dict["p38"]

In [ ]:
df_eng = df[df["engine"].isin([eng])]
new_df_failed = df_eng[df_eng["perturbation"].isin(failed_perts)]
new_df_not_failed = df_eng[~df_eng["perturbation"].isin(failed_perts)]

In [ ]:
failed_lomap_score_avg = np.average(new_df_failed["score"])
not_failed_lomap_score_avg = np.average(new_df_not_failed["score"])
print(
    f"avg score of failed is {failed_lomap_score_avg} and not failed avg score is {not_failed_lomap_score_avg}"
)

In [ ]:
# histogram of failed run in terms of perturbing atoms

df_has = df[df["percen_overlap_okay"] >= 0]
df_none = (
    pd.merge(df_has, df, how="outer", indicator=True)
    .query("_merge != 'both'")
    .drop("_merge", axis=1)
    .reset_index(drop=True)
)
print(len(df_none))
df_none["perturbing_atoms"].plot.hist(bins=10)
plt.xlabel("perturbing atoms")

In [ ]:
df_plot = df.dropna()

df_plot.plot.scatter("score", "too_small_avg", c="perturbing_atoms", colormap="viridis")
# plt.title(f"")
# plt.xlabel("lomap score")
# plt.ylabel("average no. of too small off-diagonals per leg")

df_plot.plot.scatter(
    "score", "percen_overlap_okay", c="perturbing_atoms", colormap="viridis"
)
# plt.title(f"")
# plt.xlabel("lomap score")
# plt.ylabel("percentage of okay overlap")
# )

df_plot.plot.scatter("score", "perturbing_atoms", c="too_small_avg", colormap="viridis")

In [ ]:
# plotting logarithmically
df_plot = df.dropna()

plt.xscale("log")
plt.scatter(df_plot["score"], df_plot["too_small_avg"], c=df_plot["perturbing_atoms"])
cbar = plt.colorbar()
cbar.set_label("perturbing_atoms")
plt.ylabel("too_small_avg")
plt.xlabel("score")

In [ ]:
# plotting logarithmically
df_plot = df.dropna()

# plt.xscale("log")
plt.yscale("log")

plt.scatter(
    df_plot["perturbing_atoms"],
    df_plot["too_small_avg"],
    c=df_plot["percen_overlap_okay"],
)
cbar = plt.colorbar()
cbar.set_label("percen_overlap_okay")
plt.ylabel("too_small_avg")
plt.xlabel("perturbing_atoms")

In [ ]:
# plotting logarithmically
df_plot = df.dropna()

plt.yscale("log")
plt.scatter(
    df_plot["perturbing_atoms"],
    df_plot["percen_overlap_okay"],
    c=df_plot["too_small_avg"],
)
cbar = plt.colorbar()
cbar.set_label("too_small_avg")
plt.ylabel("percen_overlap_okay")
plt.xlabel("perturbing_atoms")

In [ ]:
# plotting logarithmically
df_plot = df.dropna()

plt.yscale("log")
plt.scatter(
    df_plot["perturbing_atoms"], df_plot["diff_to_exp"], c=df_plot["too_small_avg"]
)
cbar = plt.colorbar()
cbar.set_label("too_small_avg")
plt.ylabel("diff_to_exp")
plt.xlabel("perturbing_atoms")

In [ ]:
df_plot = df.dropna()

df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c="too_small_avg", colormap="viridis"
)
# plt.title(f"")
# plt.xlabel("no. of perturbing atoms")
# plt.ylabel("percentage of okay overlap")

In [ ]:
df_plot.plot.scatter(
    "perturbing_atoms", "too_small_avg", c="percen_overlap_okay", colormap="viridis"
)
# plt.title(f"")
# plt.xlabel("no. of perturbing atoms")
# plt.ylabel("average no. of too small off-diagonals per leg")

In [ ]:
df_plot.plot.scatter(
    "percen_overlap_okay", "too_small_avg", c="perturbing_atoms", colormap="viridis"
)
# plt.title(f"")
# plt.xlabel("percentage of okay overlap")
# plt.ylabel("average no. of too small off-diagonals per leg")

In [ ]:
# exclude outliers as needed
df3 = df_plot[df_plot["diff_to_exp"] >= 20]
df_out = (
    pd.merge(df3, df_plot, how="outer", indicator=True)
    .query("_merge != 'both'")
    .drop("_merge", axis=1)
    .reset_index(drop=True)
)

# df_out = df_plot

df_out.plot.scatter(
    "perturbing_atoms", "diff_to_exp", c="percen_overlap_okay", colormap="viridis"
)

In [ ]:
columns = ["perturbing_atoms", "percen_overlap_okay", "too_small_avg"]
bins = [6, 3, 5]
for column, bin in zip(columns, bins):
    fig = plt.figure()
    df_plot = df[column]
    df_plot.plot.hist(subplots=True, bins=bin)
    plt.title(column)

In [ ]:
# for per engine
eng = "GROMACS"
df2 = df[df["engine"] == eng]
df_plot = df2.dropna()
df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c="too_small_avg", colormap="viridis"
)
df_plot.plot.scatter(
    "perturbing_atoms", "too_small_avg", c="percen_overlap_okay", colormap="viridis"
)
df_plot.plot.scatter(
    "percen_overlap_okay", "too_small_avg", c="perturbing_atoms", colormap="viridis"
)

In [ ]:
engs = ["AMBER", "SOMD", "GROMACS"]
eng_dict = {}

col_dict = pipeline.analysis.plotting_engines.set_colours()

for eng in engs:
    df2 = df[df["engine"] == eng]
    eng_dict[eng] = df2

for eng in engs:
    df_plot = eng_dict[eng].dropna()
    ax = df_plot.plot.scatter(
        "perturbing_atoms", "percen_overlap_okay", c=col_dict[eng]
    )

df_plot = eng_dict["AMBER"].dropna()
ax1 = df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c=col_dict["AMBER"]
)
df_plot = eng_dict["SOMD"].dropna()
ax2 = df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c=col_dict["SOMD"], ax=ax1
)
df_plot = eng_dict["GROMACS"].dropna()
ax3 = df_plot.plot.scatter(
    "perturbing_atoms", "percen_overlap_okay", c=col_dict["GROMACS"], ax=ax1
)
plt.legend(col_dict, loc="upper right")
print(ax1 == ax2 == ax3)

In [ ]:
engs = ["AMBER", "SOMD", "GROMACS"]
eng_dict = {}

col_dict = pipeline.analysis.plotting_engines.set_colours()

for eng in engs:
    df2 = df[df["engine"] == eng]
    df3 = df2[df2["diff_to_exp"] >= 5]
    df4 = (
        pd.merge(df3, df2, how="outer", indicator=True)
        .query("_merge != 'both'")
        .drop("_merge", axis=1)
        .reset_index(drop=True)
    )
    eng_dict[eng] = df4

for eng in engs:
    df_plot = eng_dict[eng].dropna()
    ax = df_plot.plot.scatter("perturbing_atoms", "diff_to_exp", c=col_dict[eng])

df_plot = eng_dict["AMBER"].dropna()
ax1 = df_plot.plot.scatter("perturbing_atoms", "diff_to_exp", c=col_dict["AMBER"])
df_plot = eng_dict["SOMD"].dropna()
ax2 = df_plot.plot.scatter(
    "perturbing_atoms", "diff_to_exp", c=col_dict["SOMD"], ax=ax1
)
df_plot = eng_dict["GROMACS"].dropna()
ax3 = df_plot.plot.scatter(
    "perturbing_atoms", "diff_to_exp", c=col_dict["GROMACS"], ax=ax1
)
plt.legend(col_dict, loc="upper right")
print(ax1 == ax2 == ax3)

checking which perts are bad overlap

In [ ]:
file = f"{main_folder}/extracted/mcl1/perturbing_overlap.dat"

eng_dict_ok = {"AMBER": None, "SOMD": None, "GROMACS": None}
eng_dict_not = {"AMBER": None, "SOMD": None, "GROMACS": None}

for engine in ["SOMD", "AMBER", "GROMACS"]:
    print(engine)
    perts_okay = []
    perts_not = []
    with open(file, "r") as f:
        for line in f.readlines():
            pert = line.split(",")[0].strip()
            overlap_okay = line.split(",")[3].strip()
            eng = line.split(",")[1].strip()

            if eng == engine:
                if overlap_okay == "100.0":  #  or overlap_okay == "50.0"
                    if pert not in perts_okay:
                        perts_okay.append(pert)
                else:
                    if pert not in perts_not:
                        perts_not.append(pert)

    # for pert in perts_not:
    #     if pert in perts_okay:
    #         perts_okay.remove(pert)

    print(len(perts_okay))
    print(perts_okay)
    print(len(perts_not))
    print(perts_not)
    print(" ")

    eng_dict_ok[engine] = perts_okay
    eng_dict_not[engine] = perts_not


both_perts = []

for pert in eng_dict_ok["AMBER"]:
    if pert in eng_dict_ok["SOMD"]:
        both_perts.append(pert)

print(len(both_perts))
print(both_perts)
print(" ")

all_perts = eng_dict_ok["SOMD"] + eng_dict_not["SOMD"]
not_okay = []
for pert in all_perts:
    if pert not in both_perts:
        not_okay.append(pert)

print(len(not_okay))
print(not_okay)
print(" ")

In [ ]:
prot = pipeline_protocol()
prot.num_lambda(16)
pert_dict = {(pert.split("~")[0], pert.split("~")[1]): "None" for pert in not_okay}
write_network(pert_dict, prot, "new_mcl1_network_both2.dat")